# 处理缺失值

本节介绍一些处理缺失值的通用规则，`Pandas` 对缺失值的表现形式，并演示 `Pandas` 自带的几个处理缺失值的工具的用法。  
本节以及全书涉及的缺失值主要有三种形式：`null`、`NaN` 或 `NA`。

In [1]:
import numpy as np
import pandas as pd

In [2]:
%%html
<style>
  table {margin-left: 0 !important;}
  img:nth-child(1) {width:30%; height: 30%;}
</style>

## 1. 选择处理缺失值的方法

在数据表或 `DataFrame` 中有很多识别缺失值的方法。一般情况下可以分为两种：  
一种方法是通过一个覆盖全局的掩码表示缺失值，另一种方法是用一个标签值（sentinel value）表示缺失值。  
在掩码方法中，掩码可能是一个与原数组维度相同的完整布尔类型数组，也可能是用一个比特（0 或 1）表示有缺失值的局部状态。  
在标签方法中，标签值可能是具体的数据（例如用 -9999 表示缺失的整数），也可能是些极少出现的形式。  
另外，标签值还可能是更全局的值，比如用 `NaN`（不是一个数）表示缺失的浮点数，它是 `IEEE` 浮点数规范中指定的特殊字符。  
使用这两种方法之前都需要先综合考量：  
使用单独的掩码数组会额外出现一个布尔类型数组，从而增加存储与计算的负担；  
而标签值方法缩小了可以被表示为有效值的范围，可能需要在 `CPU` 或 `GPU` 算术逻辑单元中增加额外的（往往也不是最优的）计算逻辑。  
通常使用的 `NaN` 也不能表示所有数据类型。  
大多数情况下，都不存在最佳选择，不同的编程语言与系统使用不同的方法。  
例如，`R` 语言在每种数据类型中保留一个比特作为缺失数据的标签值，而 `SciDB` 系统会在每个单元后面加一个额外的字节表示 `NA` 状态。

## 2. Pandas的缺失值

`Pandas` 里处理缺失值的方式延续了 `NumPy` 程序包的方式，并没有为浮点数据类型提供内置的 `NA` 作为缺失值。  
`Pandas` 原本也可以按照 `R` 语言采用的比特模式为每一种数据类型标注缺失值，但是这种方法非常笨拙。  
`R` 语言包含 4 种基本数据类型，而 `NumPy` 支持的类型远超 4 种。例如，`R` 语言只有一种整数类型，而 `NumPy` 支持 14 种基本的整数类型，可以根据精度、符号、编码类型按需选择。  
如果要为 `NumPy` 的每种数据类型都设置一个比特标注缺失值，可能需要为不同类型的不同操作耗费大量的时间与精力，其工作量几乎相当于创建一个新的 `NumPy` 程序包。  
另外，对于一些较小的数据类型（例如 8 位整型数据），牺牲一个比特作为缺失值标注的掩码还会导致其数据范围缩小。  
当然，`NumPy` 也是支持掩码数据的，也就是说可以用一个布尔掩码数组为原数组标注“无缺失值”或“有缺失值”。  
`Pandas` 也集成了这个功能，但是在存储、计算和编码维护方面都需要耗费不必要的资源，因此这种方式并不可取。  
综合考虑各种方法的优缺点，`Pandas` 最终选择用标签方法表示缺失值，包括两种 `Python` 原有的缺失值：浮点数据类型的 `NaN` 值，以及 `Python` 的 `None` 对象。  
后面我们将会发现，虽然这么做也会有一些副作用，但是在实际运用中的效果还是不错的。

### 2.1. None：Python对象类型的缺失值

`Pandas` 可以使用的第一种缺失值标签是 `None`，它是一个 `Python` 单体对象，经常在代码中表示缺失值。  
由于 `None` 是一个 `Python` 对象，所以不能作为任何 `NumPy` \/ `Pandas` 数组类型的缺失值，只能用于 '`object`' 数组类型（即由 `Python` 对象构成的数组）。

### 2.2. NaN：数值类型的缺失值

另一种缺失值的标签是 `NaN`（全称 `Not a Number`，不是一个数字），是一种按照 `IEEE` 浮点数标准设计、在任何系统中都兼容的特殊浮点数。  
请注意，`NumPy` 会为这个数组选择一个原生浮点类型，这意味着和之前的 `object` 类型数组不同，这个数组会被编译成 `C` 代码从而实现快速操作。  
你可以把 `NaN` 看作是一个数据类病毒——它会将与它接触过的数据同化。无论和 `NaN` 进行何种操作，最终结果都是`NaN`。

### 2.3. Pandas中NaN与None的差异

虽然 `NaN` 与 `None` 各有各的用处，但是 `Pandas` 把它们看成是可以等价交换的，在适当的时候会将两者进行替换。  
请注意，除了将整型数组的缺失值强制转换为浮点数，`Pandas` 还会自动将 `None` 转换为 `NaN`。

**Pandas对不同类型缺失值的转换规则**

| 类型 | 缺失值转换规则 | NA标签值 |
| -- | -- | -- |
| floating 浮点型 | 无变化 | np.nan |
| object 对象类型 | 无变化 | None 或 np.nan |
| integer 整数类型 | 强制转换为 float64 | np.nan |
| boolean 布尔类型 | 强制转换为 object | None 或 np.nan |

需要注意的是，`Pandas` 中字符串类型的数据通常是用 `object` 类型存储的。

## 3. 处理缺失值